# 任务 1a：执行文本生成

在此笔记本中，您将学习如何使用大型语言模型 (LLM) 生成电子邮件，以回复客户对支持工程师客户服务质量的负面反馈。在此笔记本中，您将根据客户之前的电子邮件生成一封感谢信电子邮件。您将使用基于 Amazon Bedrock API 与 Boto3 客户端的 Amazon Titan 模型。

此任务中使用的提示词称为零样本提示词。在零样本提示词中，您可以用简单的语言向语言模型描述任务或期望的输出。然后，该模型使用经过预训练的知识和功能来生成响应，或仅根据提供的提示词完成任务。

#### 场景
您是 AnyCompany 的客户服务经理 Bob，您的部分客户对客户服务不满意，并对客户支持工程师提供的服务给出了负面反馈。现在，您要在回复中为糟糕的服务向那些客户道歉，并重获信任。您需要在 LLM 的帮助下生成大量电子邮件，这些电子邮件要人性化，并根据客户在之前电子邮件通信中展现的情绪进行个性化定制。

## 任务 1a.1：环境设置

在此任务中，您将设置环境。

In [ ]:
#Create a service client by name using the default session.
import json
import os
import sys

import boto3
import botocore

module_path = ".."
sys.path.append(os.path.abspath(module_path))

bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", None))

## 任务 1a.2：生成文本

在此任务中，您将为 Amazon Bedrock 服务准备输入内容以生成电子邮件。

In [ ]:
# create the prompt
prompt_data = """
Command: Write an email from Bob, Customer Service Manager, AnyCompany to the customer "John Doe" 
who provided negative feedback on the service provided by our customer support 
engineer"""

In [ ]:
body = json.dumps({
    "inputText": prompt_data, 
    "textGenerationConfig":{
        "maxTokenCount":8192,
        "stopSequences":[],
        "temperature":0,
        "topP":0.9
        }
    }) 

接下来，您将使用 Amazon Titan 模型。

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i>**注意**：Amazon Titan 模型支持包含大约 4,000 个 Token 的上下文窗口，并且接受以下参数：
- `inputText`：提供给 LLM 的提示词
- `textGenerationConfig`：这些是模型在生成输出时会纳入考虑的参数。

Amazon Bedrock API 会为您提供 API `invoke_model`，该 API 接受以下参数：
- `modelId`：这是 Amazon Bedrock 中的各种基础模型的模型 ARN
- `accept`：输入请求的类型
- `contentType`：输出的内容类型
- `body`：包含提示词和配置的 JSON 字符串

有关可用的文本生成模型 ID，请参阅 [文档](https://docs.aws.amazon.com/bedrock/latest/userguide/model-ids-arns.html)。

## 任务 1a.3：调用 Amazon Titan 大型语言模型

在此任务中，您将探索模型如何基于之前创建的提示词生成输出。

### 完整输出生成

这是使用 Amazon Titan 模型并通过理解输入请求和根据对不同模式的固有理解生成的电子邮件。向 API 提出的请求是同步进行的，会等待模型生成完整输出。

In [ ]:
#invoke model
modelId = 'amazon.titan-text-express-v1' # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'
outputText = "\n"
try:

    response = bedrock_client.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
    response_body = json.loads(response.get('body').read())

    outputText = response_body.get('results')[0].get('outputText')

except botocore.exceptions.ClientError as error:
    
    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
        
    else:
        raise error


In [ ]:
# The relevant portion of the response begins after the first newline character
# Below we print the response beginning after the first occurence of '\n'.

email = outputText[outputText.index('\n')+1:]
print(email)


### 流式传输输出生成

Bedrock 还支持流式传输输出，因为输出是由模型以块形式生成的。这是通过调用模型并使用流式传输方法生成的电子邮件。`invoke_model_with_response_stream` 会返回一个 `ResponseStream`，您可以通过它来读取信息。

In [ ]:
# invoke model with response stream
output = []
try:
    
    response = bedrock_client.invoke_model_with_response_stream(body=body, modelId=modelId, accept=accept, contentType=contentType)
    stream = response.get('body')
    
    i = 1
    if stream:
        for event in stream:
            chunk = event.get('chunk')
            if chunk:
                chunk_obj = json.loads(chunk.get('bytes').decode())
                text = chunk_obj['outputText']
                output.append(text)
                print(f'\t\t\x1b[31m**Chunk {i}**\x1b[0m\n{text}\n')
                i+=1
            
except botocore.exceptions.ClientError as error:
    
    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
        
    else:
        raise error

这种流式传输响应的方法有助于快速获取模型的输出，并且允许服务在您读取输出时继续完成输出。这对于您要求模型生成较长文本段的使用案例很有帮助。之后，您可以整合生成的所有块以形成完整输出，并将其用于您的使用案例。

In [ ]:
#combine output chunks
print('\t\t\x1b[31m**COMPLETE OUTPUT**\x1b[0m\n')
complete_output = ''.join(output)
print(complete_output)


现在，您已尝试使用了 boto3 SDK，进而对 Amazon Bedrock API 有了基本了解。您看到了使用此 API 对客户负面反馈生成回复电子邮件的使用案例。

### 自行尝试
- 将提示词更改为您的特定使用案例，并评估不同模型的输出。
- 尝试不同的 Token 长度，了解服务的延迟和响应能力。
- 应用不同的提示词工程原则，获得更好的输出。

### 清理

您已完成此笔记本。要进入本实验的下一部分，请执行以下操作：

- 关闭此笔记本文件并继续执行 **Task1b_zh_cn.ipynb**。